In [2]:

from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from pmdarima.arima import auto_arima
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
import itertools
import warnings
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
#from tqdm import tqdm
#import itertools
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tpot import TPOTClassifier
from tpot import TPOTRegressor
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from textblob import TextBlob
import re
from typing import List
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import nltk
import pickle as pickle 

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import spacy as spacy 
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

c:\Users\dross\Desktop\Project\venv\Lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dross\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dross\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dross\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dross\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


LOCALLY

In [ ]:
# # # condense into one df 
# df1 = pd.read_csv('C:/Users/dross/Desktop/Project/aspergers.csv', parse_dates=True)
# df2 = pd.read_csv('C:/Users/dross/Desktop/Project/depression.csv', parse_dates=True)
# df3 = pd.read_csv('C:/Users/dross/Desktop/Project/ocd.csv', parse_dates=True)
# df4 = pd.read_csv('C:/Users/dross/Desktop/Project/ptsd.csv', parse_dates=True)
# df5 = pd.read_csv('C:/Users/dross/Desktop/Project/adhd.csv', parse_dates=True)

# df = pd.concat([df1, df2, df3, df4,df5], ignore_index=True)
# #LOWER ALL subreddit 
# df['subreddit'] = df['subreddit'].str.lower()
# #df.to_csv('Sentiment.csv',index=False)
# #wondered if removing nans improve model performance 
# df = df.dropna(subset=['body'])
# df.shape

GitHub

In [35]:
# # condense into one df 
df1 = pd.read_csv('Datasets/aspergers.csv', parse_dates=True)
df2 = pd.read_csv('Datasets/depression.csv', parse_dates=True)
df3 = pd.read_csv('Datasets/ocd.csv', parse_dates=True)
df4 = pd.read_csv('Datasets/ptsd.csv', parse_dates=True)
df5 = pd.read_csv('Datasets/adhd.csv', parse_dates=True)

df = pd.concat([df1, df2, df3, df4,df5], ignore_index=True)
#LOWER ALL subreddit 
df['subreddit'] = df['subreddit'].str.lower()
df = df.dropna(subset=['body'])
df.head()

(149679, 10)

Will be using the Vader sentiment analysis tested out amongst the other packages vader is best for social media data 

In [36]:
df = df[~df["body"].isin(["[deleted]", "[removed]"])]
df = df[df["body"].str.len() > 2]
df['body'] = df['body'].replace(to_replace=r'http\S+|www.\S+', value='', regex=True)

In [37]:
## continue cleaning

# remove specific structured placeholders
df['body'] = df['body'].replace(to_replace=r'\[deleted\]\n\n\[View Poll\]', value='', regex=True)

# remove markdown and keep only the descriptive text
df['body'] = df['body'].replace(to_replace=r'\[([^\]]+)\]\([^)]*\)', value=r'\1', regex=True)



df['body'] = df['body'].str.strip()  #whitespace
df = df[df['body'] != '']  # remove empty rows

In [38]:
## further cleaning
# remove more specific structured placeholders
df['body'] = df['body'].replace(to_replace=r'\[removed\].*?\[View Poll\]\(', value='', regex=True)
df['body'] = df['body'].replace(to_replace=r'\[.*?\]\(.*?\)', value='', regex=True)  
df['body'] = df['body'].replace(to_replace=r'\[|\]', value='', regex=True)  

# remove entries that only consist of punctuation or are extremely short after previous cleanings
df['body'] = df['body'].replace(to_replace=r'^\s*\W*\s*$', value='', regex=True)


df['body'] = df['body'].str.strip()  # remove whitespace
df = df[df['body'] != '']  # remove empty rows

In [39]:
# Normalize whitespace by replacing multiple spaces with a single space
df['body'] = df['body'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())


The Power of the vader is it works well if we leave stop words and caps cans be used as context so no lowering will take place or removal of stop words 

In [40]:
def clean_text_minimal(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
    text = re.sub(r'\@\w+', '', text)  # Remove mentions
    return text

sia = SentimentIntensityAnalyzer()

# Minimal cleaning
df['clean_body'] = df['body'].apply(clean_text_minimal)

# Apply VADER sentiment analysis
df['vader_sentiment'] = df['clean_body'].apply(lambda text: sia.polarity_scores(text)['compound'])

In [34]:
df.head()

,author,body,created_utc,id,num_comments,score,subreddit,title,upvote_ratio,url,target,clean_body,vader_sentiment
0,Adadum,i wish i could do a poll for this but what wou...,2021-12-23T17:56:36.000Z,rn19yc,0,1,aspergers,Separatism for high-functioning Autists?,1.0,https://www.reddit.com/r/aspergers/comments/rn...,1,i wish i could do a poll for this but what wou...,0.2144
1,Lazy_Opposite8263,i was diagnosed with high functioning asperger...,2021-12-23T17:53:41.000Z,rn17lw,1,1,aspergers,Friend blaming toxic behaviour on autism? How ...,1.0,https://www.reddit.com/r/aspergers/comments/rn...,1,i was diagnosed with high functioning asperger...,-0.9447
2,LockedOutOfElfland,it's been a while since i've seen a therapist ...,2021-12-23T17:28:07.000Z,rn0n6k,2,3,aspergers,Do therapists/mental health professionals misi...,1.0,https://www.reddit.com/r/aspergers/comments/rn...,1,it's been a while since i've seen a therapist ...,-0.9102
3,PlazmaSkittles,new here. 22 years old. diagnosed at 12. adhd ...,2021-12-23T16:54:23.000Z,rmzvwn,3,1,aspergers,I think I'm broken,1.0,https://www.reddit.com/r/aspergers/comments/rm...,1,new here. 22 years old. diagnosed at 12. adhd ...,-0.4454
4,xime_rollin,based on your understanding,2021-12-23T16:53:43.000Z,rmzvds,0,1,aspergers,I recently took an eq test and scored 20 out o...,1.0,https://www.reddit.com/r/aspergers/comments/rm...,1,based on your understanding,0.0000


Exported sample size of post for external regex validation 